In [ ]:
pip install -q yahoo_fin

In [ ]:
pip install --upgrade yfinance

In [3]:
mkdir -p data results log

In [4]:
from google.colab import drive
drive.mount('/content/drive')
! find /content/drive/MyDrive/colab/ -type f -mtime +180 -delete -print


Mounted at /content/drive
/content/drive/MyDrive/colab/results/global_0425_predictions.csv


In [5]:
import numpy as np
import pandas as pd

In [6]:
def show_prediction(name, suffix):
  df = pd.read_csv(f'results/{name}_{suffix}_predictions.csv')
  df['predicted_return'] = ((df['predicted_max']+df['predicted_min']+df['predicted_close'])/3 - df['current_price'])/df['current_price']
  return df.sort_values('predicted_return', ascending=False, key=abs)


In [7]:
def run_command_with_check(command):
  """Runs a command, prints stderr, and raises an exception if it fails."""
  process = subprocess.run(command, shell=True, capture_output=True, text=True, check=False)

  if process.returncode != 0:
    print("stderr:", process.stderr)  # Print stderr only on error
    raise subprocess.CalledProcessError(process.returncode, process.args, output=process.stdout, stderr=process.stderr)
  else:
    # Print or process stdout if needed
    #print("stdout:", process.stdout)
    pass

  return process.returncode


In [8]:
import os
import subprocess
def run_model_nn_cmd(prefix, version, do_train=False, window_size=None, future_days=None, hourly=False, copy_tickers=True):
    # copy drive/MyDrive/colab/results/nnreturn.py to local folder
    tickers = f"{prefix}tickers.csv"
    run_command_with_check(f"cp  /content/drive/MyDrive/colab/nnreturn.py .")
    if copy_tickers:
      run_command_with_check(f"cp  /content/drive/MyDrive/colab/tickers/{tickers} .")
    if not do_train:
      run_command_with_check(f"cp  /content/drive/MyDrive/colab/models/{prefix}_{version}_*.pth .")
      # Run nnreturn.py in results folder and throw exception if error
      today = pd.to_datetime('today').strftime('%m%d')
      run_command_with_check(f"rename 's/{version}/{today}/' *.pth")
      cmd_arg = ""
      if window_size:
        cmd_arg += f" --window-size {window_size}"
      if future_days:
        cmd_arg += f" --future-days {future_days}"
      if hourly:
        cmd_arg += " --hourly"
      run_command_with_check(f"python3 nnreturn.py --tickers-file {prefix}tickers.csv --mode analyze {cmd_arg}")
      run_command_with_check(f"cp {prefix}*predictions.csv results")
      # Get today's date

      return show_prediction(prefix, today)
    else:
      raise Exception("Not implemented")


In [ ]:
# 0716 is hourly 20
run_model_nn('idx', '0716', window_size=20, future_days=8, hourly=True).head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
23,XLB,89.61,2025-07-17,2025-07-19,89.62,90.66,91.40,0.010601
31,XLV,132.58,2025-07-17,2025-07-19,132.34,134.13,134.68,0.008573
20,URA,40.29,2025-07-17,2025-07-19,39.80,40.24,39.87,-0.007942
4,EEM,49.12,2025-07-17,2025-07-19,48.63,48.96,48.71,-0.007193
9,IXP,110.10,2025-07-17,2025-07-19,109.92,111.52,110.81,0.005904
14,JXI,73.21,2025-07-17,2025-07-19,72.43,73.40,72.53,-0.005782
18,SMH,292.19,2025-07-17,2025-07-19,292.37,295.67,293.22,0.005350
13,JETS,25.00,2025-07-17,2025-07-19,24.92,25.34,25.07,0.004400
11,IWO,293.85,2025-07-17,2025-07-19,292.41,296.41,296.18,0.003914
12,IYZ,29.77,2025-07-17,2025-07-19,29.50,29.77,29.72,-0.003583


In [ ]:
run_model_nn('etf', '0202').head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
4,DXYZ,28.12,2025-10-08,2025-11-05,27.48,32.79,30.08,0.071005
19,URA,51.04,2025-10-08,2025-11-05,47.80,51.07,48.98,-0.034417
29,XLV,144.55,2025-10-08,2025-11-05,143.35,155.09,149.21,0.032284
30,XLY,235.99,2025-10-08,2025-11-05,233.42,262.44,234.57,0.031725
27,XLP,77.97,2025-10-08,2025-11-05,76.30,81.83,81.68,0.025223
11,IWM,245.75,2025-10-08,2025-11-05,240.96,275.72,238.97,0.024958
25,XLI,155.24,2025-10-08,2025-11-05,154.00,169.05,154.03,0.024392
15,JXI,79.81,2025-10-08,2025-11-05,79.10,82.46,82.54,0.019505
16,QQQ,608.49,2025-10-08,2025-11-05,602.99,617.15,639.18,0.018543
9,IXN,106.04,2025-10-08,2025-11-05,104.73,108.40,110.58,0.017572


In [ ]:
# pip install  git+https://github.com/ahsank/runml#egg=runml

In [ ]:
import runml.pipeline
from runml import pipeline,findata
#from importlib import reload
# reload(runml.pipeline)

In [ ]:
# Skip some days from consecutive window to lessen test /train overlap
# findata.G_NON_OVERLAP= 0

Run following if like to save results in drive

In [ ]:
import numpy as np
# profit factor 2 = 50% of predicted gain due to early profit taking
def addAlloc(df, stop_loss, profit_factor=2):
  df['Alloc'] = df['Accu']/stop_loss - (1-df['Accu'])*profit_factor/abs(df['Gain'])
  df['Alloc'] = np.where(df['Alloc'] < 0, 0, df['Alloc'])
  return df

# profit factor 2 = 50% of predicted gain due to early profit taking
def addAllocHL(df, stop_loss, profit_factor=1):
    # Simplify following using apply
    def calculate_alloc(row):
        gain_low = min(row['Gain_l'], 0)
        gain_high = max(row['Gain_h'], 0)
        gain = min(row['Gain'], gain_high)
        average_gain = (gain_low+gain_high + 2*gain)/4
        if (average_gain == 0):
          return 0
        avg_accuracy = (row['Accu']+row['Accu_l']+row['Accu_h'])/3
        alloc =  avg_accuracy/stop_loss - (1-avg_accuracy)*profit_factor/abs(average_gain)
        return round(max(alloc, 0), 2)
    df['Alloc'] = df.apply(calculate_alloc, axis=1)
    return df

pd.options.display.max_columns = None

In [ ]:
import os
# findata.EPOCHS=200
def run_model(prefix, version, tickers, do_train):
  if (not do_train):
    os.system(f"cp  /content/drive/MyDrive/colab/results/{prefix}-{version}-* ./results/")
  mod = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(),
                         pipeline.AddMA(200), pipeline.Adj()]))
  df= pipeline.runModelCombinedVola(tickers, f"{prefix}-{version}", mod, do_train)
  # Save datafrme to prefix-version-prediction.csv
  df.to_csv(f"results/{prefix}-{version}-prediction.csv")
  return df



In [ ]:
findata.EPOCHS=200
pipeline.IS_VERBOSE = False
findata.G_SCALER = 'minmax' # Change to minmax
tickers = ['ABNB', 'ACLS' ,'AI', 'AMBA', 'APP',
           'BASE', 'BILL', 'BMBL', 'BYRN', 'CELH', 'CFLT', 'COHR', 'CRWD', 'CVNA',
           'DASH', 'DBX', 'DDOG', 'DOCN', 'DOCS', 'DOCU', 'DT', 'DXCM',
           'ELF', 'ENPH', 'ESTC', 'ETSY', 'EXPE', 'FIVE', 'FOUR',
           'GFS', 'GTLB', 'GLBE', 'HCP', 'HIMS', 'HOOD', 'HUBS',
           'INMD', 'INTA', 'IOT', 'IRDM', 'JAKK', 'JKS', 'KD', 'LYFT',
           'MBLY', 'MDB',  'MNDY', 'MNST', 'MPWR', 'MRVL', 'MXL','MTCH', 'MU',
           'NET', 'NVCR', 'NTNX', 'OKTA', 'OLED',
           'PAR', 'PANW',  'PAYX', 'PD', 'PI', 'PINS',
           'RBLX', 'RKLB', 'RMBS', 'RUM', 'SMCI', 'SMAR', 'SNAP', 'SNOW', 'SQ', 'SPOT',
           'SOUN', 'TEM', 'TEAM', 'TDOC', 'TMDX', 'TNDM', 'TOST', 'TRIP', 'TTD', 'TWLO',
           'U','UI', 'UBER', 'UPWK', 'WOLF', 'VEEV', 'Z', 'ZM', 'ZS']
df = run_model('ipos', '2a', tickers, True)

In [ ]:
addAllocHL(df, 0.10, 1)
df[(df.Gain>0)].sort_values('Alloc', ascending=False).head(50)

In [ ]:
findata.EPOCHS=200
findata.G_SCALER = 'standard'
# https://seekingalpha.com/etfs-and-funds/etf-tables/sectors
tickers1 = ['ARKF', 'ARKK', 'ARKW', 'CIBR', 'DAPP', 'DIA', 'DTEC', 'DXYZ',
            'EEM', 'FDN', 'FPX',
            'ICLN', 'IGV', 'IJK', 'IJR', 'IPO',
            'IXC', 'IXN', 'IXP', 'IWM', 'IWO', 'IYZ',
            'JETS', 'JXI', 'MGK', 'MGV', 'MTUM',
            'ONLN', 'QQQ', 'SMH', 'SMOG', 'SPY', 'TDIV',
            'URA', 'VNQ', 'VT', 'VTI', 'VUG', 'WDIV', 'XITK',
            'XLB', 'XLC', 'XLE', 'XLF', 'XLI', 'XLK', 'XLP',
            'XLRE', 'XLU', 'XLV', 'XLY', 'XME', 'XNTK', 'XSW' ]
df1 = run_model('etf', '1b', tickers1, True)



In [ ]:
detail = res['ARKK'].final_df

mindate = detail['date'].min()
totalday = (detail['date'].max()-mindate).days
detail['diff'] = ((detail['date']-mindate).dt.days+1)/totalday
detail['sum_profit'] = (detail['buy_profit'] + detail['sell_profit'])/detail['adjclose']
detail['predicted_profit'] = ((detail['adjclose_15']-detail['adjclose'])/detail['adjclose']).abs()
profit_df = detail[detail['sum_profit'] > 0][['sum_profit', 'predicted_profit']]
average_profit = (profit_df['sum_profit']).mean()
excess_profit = (profit_df['sum_profit'] - profit_df['predicted_profit']).mean()
average_loss = -detail[detail['sum_profit'] < 0]['sum_profit'].mean()
detail['has_profit'] = np.where(detail['sum_profit'] > 0, 1 , 0)

accuracy = detail['has_profit'].mean()
weight_accuracy = (detail['has_profit']*detail['diff']).sum()/detail['diff'].sum()
(accuracy, weight_accuracy, average_profit, excess_profit, average_loss)

In [ ]:
addAlloc(df1, 0.10, 1)
df1.sort_values('Alloc', ascending=False)

In [ ]:
df1.to_csv('results/etf_12b_predictions.csv')

In [ ]:
findata.EPOCHS=200
pipeline.IS_VERBOSE = False
findata.G_SCALER = 'minmax'
tickers3 = [ 'AIRI', 'ALSMY', 'AMSC', 'AMSSY', 'ACHR',
            'APPS',  'AUR',  'ATOM', 'AWRE', 'AXTI',
            'BKKT', 'BGSF', 'BLDE', 'BLNK',
            'CARS', 'CHGG', 'CLOV', 'CRCT', 'CXM',
            'DLO', 'DM', 'EVGO', 'FSLY',
            'GOGO', 'GRAB', 'GRPN', 'GTE', 'HIMX', 'HIVE',
            'INDI', 'INLX',  'INVZ', 'JOBY',  'KIND',
            'LAZR', 'LFMD', 'LUMN', 'MAX', 'MITK', 'MTTR', 'MYTE',
            'NEPH', 'NNOX', 'OMQS', 'ONDS', 'OPEN', 'OTLY', 'OUST',
            'PAYO', 'PGY', 'PLUG', 'PTON', 'PUBM',
            'REAL', 'REI', 'RIG', 'RVYL',
            'SABR', 'SFIX', 'SLDP', 'SLE', 'SMWB', 'SST', 'STEM',
            'TDW', 'TDOC', 'TRUP', 'VTEX', 'VSAT', 'YEXT']
df3 = run_model('vols', '12b', tickers3, False)

In [ ]:
# df3.sort_values('Gain', ascending=False)
addAlloc(df3, 0.17, 1)
df3[(df3.Gain>0)].sort_values('Alloc', ascending=False).head(50)

In [ ]:
findata.EPOCHS=200
tickers4 = ['AAPL', 'ACN', 'ADBE', 'AMD', 'AMZN', 'AVGO', 'BA', 'BKNG', 'BRK-B',
            'CAT', 'CDNS', 'CRM', 'COST', 'CSCO', 'DE', 'DELL', 'DHR', 'DIS',
            'GOOGL', 'FDX', 'HD', 'INTC', 'IBM', 'ISRG', 'KO', 'LLY',
            'META', 'MMC', 'MSFT', 'NKE', 'NVDA', 'NOW',
            'ORCL', 'PYPL', 'QCOM', 'SBUX',
            'TMO', 'TSLA', 'TXN', 'UNH', 'UPS', 'WMT']
df4 = run_model('mcap', '1b', tickers4, True)

In [ ]:
# df4.sort_values('Gain', ascending=False)
addAlloc(df4, 0.14, 1)
df4.sort_values('Alloc', ascending=False)

In [ ]:
findata.EPOCHS=200
pipeline.IS_VERBOSE = False
tickers5 = ['ASML', 'ADDDF', 'ADYEY',
            'BAK', 'BASFY', 'BESIY',
            'CAAP', 'CDUAF', 'COTY', 'COVTY', 'CDMGF', 'CX', 'DBOEY', 'DNNGY',
            'EADSF', 'EIFZF', 'EDNMF', 'ENGGY',
            'EQNR', 'ERIC', 'ERJ','ESEA', 'EVKIY',
            'FANUY', 'FDRVF', 'FQVLF', 'GLCNF',  'GLOB', 'HCMLY', 'HDELY',
            'HENKY', 'HYMTF', 'HMC', 'KNNGF', 'MELI', 'MGA',
            'RACE', 'LIN', 'LYB', 'LVMUY',
            'NHYDY', 'NOK', 'NU', 'NVO', 'NXPI',
            'ONON', 'OTGLY', 'PAX', 'POAHF', 'PROSF',
            'RNMBF', 'RYCEF', 'SAFRF', 'SFTBY',
            'SAP', 'SE', 'SIEGY', 'STLA',
            'TEF', 'TELNF', 'TGLS','THQQF', 'TKAMY', 'TRYIY', 'TSM',
            'VALE', 'VNT', 'YARIY', 'ZURVY']

df5 = run_model('gstock', '2a', tickers5, False)

In [ ]:
addAllocHL(df5, 0.14)
df5[df5.Gain > 0].sort_values('Alloc', ascending=False).head(50)

In [ ]:
findata.G_SCALER = 'standard'
tickers6 = ['ARGT', 'EWD', 'FLN',
           'ECH', 'EPHE', 'EWA', 'EWG', 'EWI', 'EWJ',
           'EWC', 'EWM', 'EWP', 'EWT', 'EWW', 'EWY', 'EWZ',
           'IDX', 'ILF', 'INDA', 'KEMQ', 'KWT', 'THD', 'TUR', 'VNM', 'XCEM']
df6 = run_model('etf', '12a', tickers6, False)

In [ ]:
addAllocHL(df6, 0.07, 1)
df6.sort_values('Alloc', ascending=False)

In [ ]:
findata.EPOCHS=200
pipeline.IS_VERBOSE = False
findata.G_SCALER = 'standard'
tickers7 = ['AAL', 'ADI', 'ALB', 'AMR','ANF', 'APO', 'AQN', 'ARCH', 'ARE', 'ASC',
            'BAH', 'BAX', 'BKR', 'BLK', 'BX', 'BXC',
            'CBT', 'CC', 'CCI', 'CCJ', 'CF', 'CI', 'CIVI', 'CTAS',
            'CG',  'CLS', 'CMI', 'COKE', 'COP', 'CSL',
            'DAL', 'DD', 'DINO', 'DVN', 'DXC', 'EMN',
            'F', 'FCX', 'FICO', 'FIS', 'FLO', 'FMS', 'FSLR', 'GLW', 'GWW',
            'HIW', 'HSII', 'HPE', 'IFF', 'IP', 'IPI', 'IVZ',
            'KD', 'KHC', 'KMB', 'KVUE',
            'MMM', 'MOD', 'MPC', 'NFG', 'NTR', 'OMC', 'OLN', 'OSK',
            'PARA', 'PCAR', 'PEP', 'PETS', 'PNR', 'POR', 'POWL', 'PSTL',
            'RCL', 'RH', 'ROP',
            'SAFE', 'SBLK', 'SCCO', 'SPTN', 'SPG', 'SRE', 'STLD', 'STRL', 'SWK',
            'T', 'TEX', 'TGT', 'TDG', 'TMUS','TWI', 'UAL', 'URI',
            'VRTX', 'VZ', 'WDC', 'WHR', 'WSM', 'XOM', 'YORW', 'ZTS']
df7 = run_model('val', '1a', tickers7, False)

In [ ]:
addAllocHL(df7, 0.10, 1)
df7[df7.Gain > 0].sort_values('Alloc', ascending=False).head(50)

In [ ]:
findata.EPOCHS=200
tickers8 = [ 'DBC', 'FXB', 'FXE', 'FXF', 'FXY',
            'GLD', 'ISHG', 'PDBC', 'SLV', 'SOYB',
            'TIP', 'TLT', 'USO', 'UUP', 'VTIP', 'WEAT']

df8 =  run_model('com', '12a', tickers8, True)

In [ ]:
addAllocHL(df8, 0.10, 1)
df8.sort_values('Alloc', ascending=False)

In [ ]:
run_model_nn('pennystock', '0630').head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
60,SMWB,8.98,2025-10-09,2025-11-06,7.51,12.06,14.34,0.258723
52,REAL,10.10,2025-10-09,2025-11-06,7.90,10.44,7.96,-0.132013
18,CRCT,5.56,2025-10-09,2025-11-06,3.79,5.63,5.08,-0.130695
43,ONDS,10.98,2025-10-09,2025-11-06,8.43,11.08,9.31,-0.125076
44,OPEN,8.22,2025-10-09,2025-11-06,6.03,8.51,7.10,-0.122466
41,NEPH,5.31,2025-10-09,2025-11-06,4.15,5.40,4.44,-0.121783
34,JOBY,17.08,2025-10-09,2025-11-06,12.99,17.35,14.82,-0.118657
5,AUR,5.45,2025-10-09,2025-11-06,5.16,5.82,7.30,0.118043
67,VSAT,34.90,2025-10-09,2025-11-06,29.07,35.71,29.97,-0.095033
10,BKKT,37.01,2025-10-09,2025-11-06,30.58,36.85,33.26,-0.093128


In [ ]:
findata.EPOCHS=200
findata.G_SCALER = 'standard'
tickers9 = [ 'ADA-USD', 'AVAX-USD', 'BTC-USD', 'BCH-USD', 'DOGE-USD', 'DOT-USD',
            'ETH-USD', 'FIL-USD', 'ICP-USD', 'LINK-USD', 'LTC-USD',
            'MATIC-USD', 'NEAR-USD', 'SOL-USD', 'TON-USD', 'TRX-USD', 'XRP-USD']
df9 = run_model('cry', '11b', tickers9, False)

In [ ]:
addAllocHL(df9, 0.14, 1)
df9.sort_values('Alloc', ascending=False)

In [ ]:
run_model_nn('bluechipstock', '0221').head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
35,SBUX,81.03,2025-10-10,2025-11-07,81.01,87.93,87.58,0.055247
2,ADBE,344.44,2025-10-10,2025-11-07,335.08,374.99,370.14,0.045378
22,IBM,285.18,2025-10-10,2025-11-07,276.89,300.14,315.66,0.043423
27,MMC,206.18,2025-10-10,2025-11-07,205.28,222.39,217.64,0.043279
40,UPS,85.43,2025-10-10,2025-11-07,84.51,91.49,90.12,0.038355
41,WMT,102.45,2025-10-10,2025-11-07,101.21,110.23,107.46,0.037579
1,ACN,244.34,2025-10-10,2025-11-07,237.73,263.15,257.47,0.034556
23,ISRG,442.06,2025-10-10,2025-11-07,421.91,486.53,463.42,0.034445
8,BRK-B,494.06,2025-10-10,2025-11-07,480.87,527.32,523.08,0.033120
15,DELL,157.34,2025-10-10,2025-11-07,152.92,171.96,162.32,0.032160


In [ ]:
run_model_nn('country', '0224').head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
3,ECH,31.67,2025-04-25,2025-05-23,29.52,32.54,29.63,-0.034944
7,EWI,44.04,2025-04-25,2025-05-23,40.22,45.26,42.10,-0.034363
11,EWP,40.86,2025-04-25,2025-05-23,36.79,42.01,41.18,-0.021211
17,ILF,24.90,2025-04-25,2025-05-23,23.96,25.83,23.37,-0.020616
21,THD,53.11,2025-04-25,2025-05-23,52.15,56.15,54.28,0.020398
6,EWG,39.36,2025-04-25,2025-05-23,37.23,40.48,38.05,-0.019648
4,EPHE,26.26,2025-04-25,2025-05-23,26.09,27.18,26.96,0.018406
9,EWC,42.04,2025-04-25,2025-05-23,41.29,43.43,43.39,0.015779
20,KWT,35.71,2025-04-25,2025-05-23,34.17,36.68,34.69,-0.014842
2,FLN,18.61,2025-04-25,2025-05-23,17.94,19.28,17.79,-0.014687


In [ ]:
run_model_nn('global', '0225').head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
40,LOMA,8.31,2025-10-14,2025-11-11,8.12,11.65,10.81,0.226635
28,GLOB,59.34,2025-10-14,2025-11-11,59.12,68.23,70.37,0.110662
3,BAK,2.43,2025-10-14,2025-11-11,2.44,2.77,2.71,0.086420
49,PAX,14.28,2025-10-14,2025-11-11,12.56,14.31,12.82,-0.073529
15,DNNGY,21.79,2025-10-14,2025-11-11,20.57,25.06,24.30,0.069757
55,SAP,269.70,2025-10-14,2025-11-11,266.27,298.69,294.99,0.062848
16,EADSF,233.75,2025-10-14,2025-11-11,210.21,236.07,218.72,-0.051693
6,BESIY,165.34,2025-10-13,2025-11-10,150.40,170.76,152.25,-0.045583
62,TKAMY,14.60,2025-10-14,2025-11-11,13.06,14.66,14.10,-0.045205
65,VALE,10.94,2025-10-14,2025-11-11,9.65,11.16,10.55,-0.044485


In [ ]:
run_model_nn_cmd('valuestock', '0301').head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
74,RH,174.27,2025-10-17,2025-11-14,166.78,188.88,205.02,0.072435
22,CTAS,188.10,2025-10-17,2025-11-14,183.68,205.98,209.10,0.061067
66,PEP,153.71,2025-10-17,2025-11-14,153.15,173.49,161.76,0.059137
17,CCI,97.48,2025-10-17,2025-11-14,95.47,110.40,103.46,0.057755
26,COKE,131.68,2025-10-17,2025-11-14,133.60,144.78,139.38,0.057513
0,AAL,11.86,2025-10-17,2025-11-14,11.57,13.99,12.06,0.057336
93,VRTX,416.81,2025-10-17,2025-11-14,404.17,463.54,453.00,0.056205
36,ENTG,90.94,2025-10-17,2025-11-14,82.68,92.97,82.13,-0.055128
23,CG,57.14,2025-10-17,2025-11-14,51.46,59.02,51.75,-0.053611
72,PSTL,15.05,2025-10-17,2025-11-14,14.63,16.18,16.73,0.052935


In [ ]:
run_model_nn_cmd('commodity', '0312').head(5)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
5,GLD,380.61,2025-10-21,2025-11-18,370.06,426.80,387.69,0.037414
14,VTIP,50.19,2025-10-21,2025-11-18,48.90,54.72,49.64,0.017865
15,WEAT,4.04,2025-10-21,2025-11-18,3.95,4.21,4.09,0.010726
13,UUP,27.90,2025-10-21,2025-11-18,27.23,29.08,28.26,0.010394
8,SLV,44.02,2025-10-21,2025-11-18,42.89,46.25,44.19,0.009617


In [ ]:
run_model_nn_cmd('crypto', '0214').head(5)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
3,ADA-USD,0.67,2025-10-21,2025-11-18,0.60,0.90,0.75,0.119403
7,FIL-USD,1.61,2025-10-21,2025-11-18,1.61,1.75,1.95,0.099379
5,DOT-USD,3.13,2025-10-21,2025-11-18,2.86,3.82,3.62,0.096912
12,AVAX-USD,20.58,2025-10-21,2025-11-18,14.94,22.87,18.43,-0.089083
10,BCH-USD,502.16,2025-10-21,2025-11-18,381.60,553.39,445.81,-0.083426


In [ ]:
run_model_nn_cmd('techstock', '0312').head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
83,ZS,307.92,2025-10-22,2025-11-19,243.20,308.87,245.14,-0.136994
15,DASH,252.31,2025-10-22,2025-11-19,213.76,250.49,221.27,-0.094342
41,LYFT,20.26,2025-10-22,2025-11-19,16.44,20.88,18.25,-0.085719
11,CFLT,22.58,2025-10-22,2025-11-19,21.57,25.89,25.70,0.080012
36,INTA,39.40,2025-10-22,2025-11-19,39.53,46.05,41.77,0.077411
61,PUBM,8.42,2025-10-22,2025-11-19,8.15,9.91,9.00,0.071259
81,Z,77.37,2025-10-22,2025-11-19,74.40,80.95,93.17,0.070699
30,GTLB,48.09,2025-10-22,2025-11-19,47.13,58.83,48.37,0.069730
28,FOUR,75.39,2025-10-22,2025-11-19,70.57,85.25,85.24,0.065835
21,DT,49.56,2025-10-22,2025-11-19,48.00,56.20,53.63,0.061542


In [ ]:
run_model_nn('stock', '0123').head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
204,PUBM,12.82,2025-07-03,2025-07-31,12.64,16.44,14.37,0.129745
375,PI,115.86,2025-07-03,2025-07-31,116.02,144.76,128.19,0.119080
17,INTA,48.11,2025-07-03,2025-07-31,45.99,55.07,59.27,0.110857
352,ABR,10.90,2025-07-03,2025-07-31,10.53,12.98,12.60,0.104281
144,LFMD,12.50,2025-07-03,2025-07-31,11.93,14.68,14.55,0.097600
378,RCAT,7.08,2025-07-03,2025-07-31,5.73,7.56,5.93,-0.095104
10,ARQT,13.82,2025-07-03,2025-07-31,13.37,16.73,15.27,0.094308
348,CRM,272.15,2025-07-03,2025-07-31,267.03,319.47,305.57,0.092620
283,NTGR,29.41,2025-07-03,2025-07-31,28.97,35.49,31.90,0.092146
150,POWL,217.66,2025-07-03,2025-07-31,204.27,250.88,256.42,0.089727


In [ ]:
run_model_nn('buystocks', '0402', False, 500, 250).head(10)
# show_prediction('buystocks', '0508').head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
101,CHGG,1.29,2025-10-14,2026-09-29,1.27,2.56,2.49,0.633075
360,QBTS,43.06,2025-10-14,2026-09-29,37.01,88.83,81.98,0.608763
102,SLP,15.26,2025-10-14,2026-09-29,14.61,29.33,29.45,0.603102
82,ETSY,71.91,2025-10-14,2026-09-29,68.53,140.91,121.81,0.535484
325,BE,114.06,2025-10-14,2026-09-29,96.53,222.55,205.95,0.534368
9,HIMS,54.02,2025-10-14,2026-09-29,45.33,101.46,101.70,0.533321
348,SH,37.23,2025-10-14,2026-09-29,36.60,65.16,69.38,0.532277
295,CRNC,13.07,2025-10-14,2026-09-29,12.71,23.51,23.62,0.526141
18,OUST,36.16,2025-10-14,2026-09-29,32.69,69.77,62.96,0.524889
253,SLDP,6.94,2025-10-14,2026-09-29,6.73,13.27,11.71,0.523055


In [10]:
import sys
sys.path.append('/content')
import stock_prediction.stock_prediction.stock_predictor_lib
from stock_prediction.stock_prediction.stock_predictor_lib import run_model_nn, show_prediction, StockPredictorLib, Config
colab_path = 'content/drive/MyDrive/colab'
tfrcfg = Config.from_file('./stock_prediction/configs/experiment.yaml')

In [ ]:
techcfg = Config.from_file('./stock_prediction/configs/experiment.yaml')
colab_path = 'content/drive/MyDrive/colab'
techcfg.model_path = f'/{colab_path}/models/transformer/techstock_1005'
techdf = run_model_nn(f'/{colab_path}/tickers/techstocktickers.csv', cfg=techcfg)

Loading models for return types: {'min': '/content/drive/MyDrive/colab/models/transformer/techstock_1005_min_model.pth', 'max': '/content/drive/MyDrive/colab/models/transformer/techstock_1005_max_model.pth', 'close': '/content/drive/MyDrive/colab/models/transformer/techstock_1005_close_model.pth'}


In [ ]:
techdf.head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
51,NVCR,13.86,2025-10-22,2025-11-19,13.50,18.26,19.08,0.222703
7,BILL,50.71,2025-10-22,2025-11-19,49.05,69.35,57.36,0.155328
55,PAR,37.22,2025-10-22,2025-11-19,25.63,38.41,31.02,-0.148666
8,BMBL,5.67,2025-10-22,2025-11-19,5.22,6.55,7.72,0.145797
32,HIMS,47.46,2025-10-22,2025-11-19,44.09,58.07,60.34,0.141312
11,CFLT,22.58,2025-10-22,2025-11-19,21.50,29.60,26.02,0.138471
59,PI,202.08,2025-10-22,2025-11-19,138.09,213.31,183.09,-0.118352
69,TDOC,8.34,2025-10-22,2025-11-19,6.64,9.91,5.82,-0.105915
26,ETSY,73.25,2025-10-22,2025-11-19,69.82,103.52,69.65,0.105757
79,UPWK,16.01,2025-10-22,2025-11-19,15.57,18.11,19.20,0.100979


In [ ]:
defetfcfg = Config.from_file('./stock_prediction/configs/default.yaml')
colab_path = 'content/drive/MyDrive/colab'
defetfcfg.model_path = f'/{colab_path}/models/default/etf_1008'
techdf = run_model_nn(f'/{colab_path}/tickers/etftickers.csv', cfg=defetfcfg)
techdf.head(10)

Loading models for return types: {'min': '/content/drive/MyDrive/colab/models/default/etf_1008_min_model.pth', 'max': '/content/drive/MyDrive/colab/models/default/etf_1008_max_model.pth', 'close': '/content/drive/MyDrive/colab/models/default/etf_1008_close_model.pth'}


,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
13,IYZ,33.03,2025-10-08,2025-11-05,30.87,33.49,31.85,-0.029064
6,IJR,118.99,2025-10-08,2025-11-05,108.90,122.23,116.23,-0.026921
15,JXI,79.81,2025-10-08,2025-11-05,74.77,81.14,77.11,-0.026772
20,VNQ,89.93,2025-10-08,2025-11-05,87.95,96.33,92.71,0.026687
28,XLU,90.63,2025-10-08,2025-11-05,85.39,92.23,87.12,-0.026297
30,XLY,235.95,2025-10-08,2025-11-05,230.70,251.59,242.76,0.024299
0,ARKF,58.20,2025-10-08,2025-11-05,54.67,62.53,61.62,0.024170
10,IXP,122.22,2025-10-08,2025-11-05,118.91,128.84,127.49,0.023400
23,XLE,89.17,2025-10-08,2025-11-05,81.98,92.03,88.31,-0.019401
2,ARKW,179.71,2025-10-08,2025-11-05,168.07,183.64,177.37,-0.018641


In [ ]:
tvolcfg = Config.from_file('./stock_prediction/configs/experiment.yaml')
tvolcfg.model_path = f'/{colab_path}/models/transformer/pennystock_1009'
techdf = run_model_nn(f'/{colab_path}/tickers/pennystocktickers.csv', cfg=tvolcfg)
techdf.head(10)

Loading models for return types: {'min': '/content/drive/MyDrive/colab/models/transformer/pennystock_1009_min_model.pth', 'max': '/content/drive/MyDrive/colab/models/transformer/pennystock_1009_max_model.pth', 'close': '/content/drive/MyDrive/colab/models/transformer/pennystock_1009_close_model.pth'}


ERROR:yfinance:HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"Quote not found for symbol: KIND"}}}
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['KIND']: YFTzMissingError('possibly delisted; no timezone found')
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['MYTE']: YFTzMissingError('possibly delisted; no timezone found')
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['SAVE']: YFTzMissingError('possibly delisted; no timezone found')


,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
62,STEM,23.24,2025-10-09,2025-11-06,22.29,29.74,36.95,0.276248
18,CRCT,5.55,2025-10-09,2025-11-06,3.02,5.93,3.58,-0.247447
59,SLE,3.60,2025-10-09,2025-11-06,3.27,4.88,5.29,0.244444
9,BBAI,7.20,2025-10-09,2025-11-06,4.18,8.54,4.41,-0.206944
21,EVGO,4.72,2025-10-09,2025-11-06,2.68,5.86,2.70,-0.206215
6,ATOM,4.81,2025-10-09,2025-11-06,4.37,8.04,4.93,0.201663
58,SLDP,6.24,2025-10-08,2025-11-05,4.60,6.96,3.45,-0.198184
35,LAZR,2.16,2025-10-09,2025-11-06,1.96,2.74,2.76,0.151235
38,MAX,10.81,2025-10-09,2025-11-06,10.58,12.10,14.42,0.144002
33,IRDM,19.58,2025-10-09,2025-11-06,19.07,23.73,24.38,0.143684


In [ ]:
bctcfg = Config.from_file('./stock_prediction/configs/experiment.yaml')
bctcfg.model_path = f'/{colab_path}/models/transformer/bluechipstock_1010'
bctdf = run_model_nn(f'/{colab_path}/tickers/bluechipstocktickers.csv', cfg=bctcfg)
bctdf.head(10)

Loading models for return types: {'min': '/content/drive/MyDrive/colab/models/transformer/bluechipstock_1010_min_model.pth', 'max': '/content/drive/MyDrive/colab/models/transformer/bluechipstock_1010_max_model.pth', 'close': '/content/drive/MyDrive/colab/models/transformer/bluechipstock_1010_close_model.pth'}


,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
33,PYPL,74.64,2025-10-10,2025-11-07,67.82,75.60,51.30,-0.130404
16,DHR,206.08,2025-10-10,2025-11-07,175.94,209.54,177.16,-0.089933
3,AMD,226.66,2025-10-10,2025-11-07,223.61,275.26,239.20,0.085429
37,TSLA,439.87,2025-10-10,2025-11-07,374.50,447.50,389.20,-0.082153
34,QCOM,163.57,2025-10-10,2025-11-07,140.74,165.47,144.84,-0.080822
7,BKNG,5200.49,2025-10-10,2025-11-07,4495.45,5289.34,4699.77,-0.071590
6,BA,217.17,2025-10-10,2025-11-07,193.58,218.44,199.56,-0.061288
17,DIS,112.14,2025-10-10,2025-11-07,97.11,113.70,106.83,-0.055823
30,NVDA,195.33,2025-10-10,2025-11-07,173.97,196.77,183.57,-0.054062
15,DELL,158.41,2025-10-10,2025-11-07,146.17,163.27,140.41,-0.053406


In [12]:
stkcfg = Config.from_file('./stock_prediction/configs/experiment.yaml')
stkcfg.model_path = f'/{colab_path}/models/transformer/stock_1012'
stkdf = run_model_nn(f'/{colab_path}/tickers/stocktickers.csv', cfg=stkcfg)

Loading models for return types: {'min': '/content/drive/MyDrive/colab/models/transformer/stock_1012_min_model.pth', 'max': '/content/drive/MyDrive/colab/models/transformer/stock_1012_max_model.pth', 'close': '/content/drive/MyDrive/colab/models/transformer/stock_1012_close_model.pth'}


In [13]:
stkdf.head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
357,RGTI,40.19,2025-10-24,2025-11-21,37.92,48.29,74.77,0.335158
362,AEVA,17.14,2025-10-24,2025-11-21,16.27,20.62,22.43,0.153637
354,ROOT,77.00,2025-10-24,2025-11-21,72.08,88.46,102.17,0.137273
26,OSCR,20.32,2025-10-24,2025-11-21,13.34,21.70,17.63,-0.135991
9,HIMS,50.25,2025-10-24,2025-11-21,45.10,53.49,71.37,0.127430
231,LEU,379.00,2025-10-24,2025-11-21,303.12,610.62,361.89,0.121926
212,MSTR,289.77,2025-10-24,2025-11-21,252.07,368.22,354.95,0.121855
308,RVLV,22.79,2025-10-24,2025-11-21,18.57,24.53,17.01,-0.120813
361,QBTS,33.12,2025-10-24,2025-11-21,30.27,37.00,20.10,-0.120672
15,CRDO,160.99,2025-10-24,2025-11-21,117.26,175.76,133.25,-0.117399


In [ ]:
glbcfg = Config.from_file('./stock_prediction/configs/experiment.yaml')
glbcfg.model_path = f'/{colab_path}/models/transformer/global_0907'
glbdf = run_model_nn(f'globaltickers.csv', cfg=glbcfg)


In [ ]:
glbdf.head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
66,VNT,39.04,2025-10-14,2025-11-11,39.34,44.33,43.83,0.088627
33,HMC,30.50,2025-10-14,2025-11-11,30.57,33.91,33.85,0.074645
32,IFNNy,37.23,2025-10-14,2025-11-11,36.66,42.45,40.29,0.069030
38,LIN,459.58,2025-10-14,2025-11-11,467.18,500.48,504.94,0.068077
46,NXPI,215.82,2025-10-14,2025-11-11,216.80,239.79,233.57,0.065950
11,CDMGF,22.64,2025-10-13,2025-11-10,22.57,24.95,24.73,0.063751
10,COVTY,34.26,2025-10-13,2025-11-10,34.51,37.81,36.39,0.057696
3,BAK,2.45,2025-10-14,2025-11-11,2.39,2.71,2.63,0.051701
9,COTY,4.28,2025-10-14,2025-11-11,4.26,4.64,4.59,0.050623
37,RACE,381.73,2025-10-14,2025-11-11,382.77,412.24,406.37,0.049066


In [ ]:
valtcfg = Config.from_file('./stock_prediction/configs/experiment.yaml')
valtcfg.model_path = f'/{colab_path}/models/transformer/valuestock_1018'
valtdf = run_model_nn(f'valuestocktickers.csv', cfg=valtcfg)


In [ ]:
valtdf.head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
44,GLW,84.49,2025-10-17,2025-11-14,81.38,115.73,113.66,0.226062
14,BXC,70.15,2025-10-17,2025-11-14,67.42,98.92,72.73,0.135994
34,EMN,61.28,2025-10-17,2025-11-14,60.12,72.23,75.40,0.130059
16,CC,12.90,2025-10-17,2025-11-14,12.54,15.84,15.23,0.126873
58,MOD,153.79,2025-10-17,2025-11-14,127.87,158.81,127.45,-0.102391
0,AAL,11.86,2025-10-17,2025-11-14,9.83,12.45,9.68,-0.101743
19,CF,83.51,2025-10-17,2025-11-14,61.47,86.64,78.17,-0.096795
13,BX,156.31,2025-10-17,2025-11-14,103.96,164.35,155.24,-0.096774
56,KVUE,15.29,2025-10-17,2025-11-14,14.43,17.82,17.97,0.094833
74,RH,174.27,2025-10-17,2025-11-14,167.53,218.79,180.49,0.084161


In [ ]:
commtcfg = Config.from_file('./stock_prediction/configs/experiment.yaml')
commtcfg.model_path = f'/{colab_path}/models/transformer/commodity_1021'
commtdf = run_model_nn(f'/{colab_path}/tickers/commoditytickers.csv', cfg=commtcfg)

Loading models for return types: {'min': '/content/drive/MyDrive/colab/models/transformer/commodity_1021_min_model.pth', 'max': '/content/drive/MyDrive/colab/models/transformer/commodity_1021_max_model.pth', 'close': '/content/drive/MyDrive/colab/models/transformer/commodity_1021_close_model.pth'}


In [ ]:
commtdf.head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
5,GLD,381.15,2025-10-21,2025-11-18,377.94,408.54,416.12,0.051729
12,USO,67.83,2025-10-21,2025-11-18,67.22,69.96,74.23,0.038921
7,PALL,128.90,2025-10-21,2025-11-18,126.66,137.55,134.23,0.030359
15,WEAT,4.04,2025-10-21,2025-11-18,3.87,4.48,4.10,0.027228
0,DBC,22.08,2025-10-21,2025-11-18,20.44,22.28,22.03,-0.022494
8,SLV,47.72,2025-10-20,2025-11-17,46.37,48.63,49.92,0.012294
13,UUP,27.90,2025-10-21,2025-11-18,27.67,28.66,28.09,0.008602
4,FXY,60.58,2025-10-21,2025-11-18,60.38,61.65,60.78,0.005888
1,FXB,128.79,2025-10-21,2025-11-18,127.56,130.69,130.12,0.005176
6,ISHG,75.58,2025-10-21,2025-11-18,74.07,76.31,75.69,-0.002955


In [ ]:
cryptcfg = Config.from_file('./stock_prediction/configs/experiment.yaml')
cryptcfg.model_path = f'/{colab_path}/models/transformer/crypto_1021'
cryptdf = run_model_nn(f'/{colab_path}/tickers/cryptotickers.csv', cfg=cryptcfg)

Loading models for return types: {'min': '/content/drive/MyDrive/colab/models/transformer/crypto_1021_min_model.pth', 'max': '/content/drive/MyDrive/colab/models/transformer/crypto_1021_max_model.pth', 'close': '/content/drive/MyDrive/colab/models/transformer/crypto_1021_close_model.pth'}


In [ ]:
cryptdf.head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
8,LTC-USD,96.01,2025-10-21,2025-11-18,92.26,142.93,95.96,0.149707
2,SOL-USD,194.53,2025-10-21,2025-11-18,183.08,220.31,240.31,0.103000
6,DOGE-USD,0.20,2025-10-21,2025-11-18,0.19,0.24,0.22,0.083333
17,MATIC-USD,0.22,2025-03-24,2025-04-21,0.15,0.26,0.20,-0.075758
11,ICP-USD,3.24,2025-10-21,2025-11-18,3.12,3.92,3.32,0.065844
12,AVAX-USD,20.75,2025-10-21,2025-11-18,19.90,25.64,20.64,0.063133
0,BTC-USD,113142.62,2025-10-21,2025-11-18,111504.18,129382.97,119838.14,0.062745
10,BCH-USD,505.01,2025-10-21,2025-11-18,477.52,547.41,572.59,0.054448
13,LINK-USD,18.85,2025-10-21,2025-11-18,18.22,22.25,18.81,0.048276
1,ETH-USD,4058.20,2025-10-21,2025-11-18,3981.90,4263.35,4354.49,0.034920


In [ ]:
from google.colab import auth
import gspread
from oauth2client.client import GoogleCredentials
import pandas as pd
from google.auth import default


auth.authenticate_user()


creds, _ = default()
gc = gspread.authorize(creds)
# gc = gspread.authorize(GoogleCredentials.get_application_default())


In [ ]:
# Open your spreadsheet by name
spreadsheet = gc.open('Allocation')
# Or by URL: spreadsheet = gc.open_by_url('Your Spreadsheet URL')

# Select the worksheet (e.g., the first sheet)
worksheet = spreadsheet.worksheet("Buy")

# Get all values as a list of lists
data = worksheet.get_all_values()

# Convert to a Pandas DataFrame
df = pd.DataFrame(data[1:], columns=data[0]) # Assuming first row is header


In [ ]:
df = df[['Ticker']]
df.to_csv('stocktickers.csv', index=False)
df.to_csv('buystockstickers.csv', index=False)

In [ ]:
# Only have Tickers column

run_model_nn('buystocks', '0402', False, 500, 250, copy_tickers=False).head(10)


,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
333,QBTS,34.25,2025-10-08,2026-09-23,32.44,71.01,65.57,0.644964
96,CHGG,1.42,2025-10-08,2026-09-23,1.40,2.82,2.74,0.633803
97,SLP,16.47,2025-10-08,2026-09-23,16.09,32.07,31.69,0.616070
18,OUST,30.65,2025-10-08,2026-09-23,29.45,61.59,56.80,0.607830
9,HIMS,58.17,2025-10-08,2026-09-23,49.10,110.63,110.40,0.547934
252,LYFT,21.16,2025-10-08,2026-09-23,19.66,41.40,36.93,0.543636
6,DOMO,14.74,2025-10-08,2026-09-23,12.98,29.59,25.17,0.531886
326,ROOT,81.73,2025-10-08,2026-09-23,76.40,156.09,141.76,0.526367
122,LZ,9.97,2025-10-08,2026-09-23,9.63,19.32,16.65,0.524574
276,CRNC,11.96,2025-10-08,2026-09-23,11.62,21.43,21.43,0.518395


In [ ]:
 # !cp -ru results /content/drive/MyDrive/colab
 !rsync -av --ignore-existing results/* /content/drive/MyDrive/colab/results/
 ! find /content/drive/MyDrive/colab/ -type f -mtime +180 -delete -print

sending incremental file list
buystocks_0709_predictions.csv

sent 18,897 bytes  received 35 bytes  37,864.00 bytes/sec
total size is 18,770  speedup is 0.99
